#### 데이터 전처리 및 시각화

##### 전처리
데이터프레임 병합

In [1]:
# import
import pandas as pd
import numpy as np

import folium
import json

In [5]:
# 스타벅스 매장리스트
sbuck_df = pd.read_excel('./busan_starbuks.xlsx', header=0)
sbuck_df.tail()

,매장명,위도,경도,매장타입,주소
132,해운대 엑스더스카이,35.159965,129.169756,general,부산광역시 해운대구 달맞이길30
133,센텀신세계3F,35.169063,129.129066,general,부산광역시 해운대구 센텀남대로 35 (우동)
134,해운대중동역,35.167185,129.167886,general,부산광역시 해운대구 좌동순환로 6 (중동)
135,센텀몰1F,35.169810,129.127783,general,부산광역시 해운대구 센텀4로 15 (우동)
136,그랜드조선 부산,35.160034,129.163051,general,부산광역시 해운대구 해운대해변로 292 (중동)


In [6]:
sbuck_df['주소'][136].split()[1]

'해운대구'

In [7]:
gu_list = []
for item in sbuck_df['주소']:
    gu = item.split()[1]
    gu_list.append(gu)

sbuck_df['구'] = gu_list
sbuck_df.tail()

,매장명,위도,경도,매장타입,주소,구
132,해운대 엑스더스카이,35.159965,129.169756,general,부산광역시 해운대구 달맞이길30,해운대구
133,센텀신세계3F,35.169063,129.129066,general,부산광역시 해운대구 센텀남대로 35 (우동),해운대구
134,해운대중동역,35.167185,129.167886,general,부산광역시 해운대구 좌동순환로 6 (중동),해운대구
135,센텀몰1F,35.169810,129.127783,general,부산광역시 해운대구 센텀4로 15 (우동),해운대구
136,그랜드조선 부산,35.160034,129.163051,general,부산광역시 해운대구 해운대해변로 292 (중동),해운대구


##### 부산시 행정구역별 위경도 전처리

In [59]:
busan_loc_df = pd.read_excel('./대한민국_행정구역_위경도.xlsx')
busan_loc_df.head()

,docity,do,city,longitude,latitude
0,강원강릉시,강원,강릉시,128.878497,37.749136
1,강원고성군,강원,고성군,128.470164,38.377961
2,강원동해시,강원,동해시,129.116633,37.521931
3,강원삼척시,강원,삼척시,129.167489,37.447086
4,강원속초시,강원,속초시,128.594167,38.204275


In [60]:
busan_gu_df = busan_loc_df[busan_loc_df['do'] == '부산시']
busan_gu_df.head()

,docity,do,city,longitude,latitude
122,부산시강서구,부산시,강서구,128.982908,35.209164
123,부산시금정구,부산시,금정구,129.094319,35.240078
124,부산시남구,부산시,남구,129.086500,35.133408
125,부산시동구,부산시,동구,129.059175,35.135894
126,부산시동래구,부산시,동래구,129.085856,35.201872


In [12]:
busan_gu_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16 entries, 122 to 137
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   docity     16 non-null     object 
 1   do         16 non-null     object 
 2   city       16 non-null     object 
 3   longitude  16 non-null     float64
 4   latitude   16 non-null     float64
dtypes: float64(2), object(3)
memory usage: 768.0+ bytes


In [13]:
busan_gu_df.to_excel('./busan_gu_info.xlsx', index=False)

##### 부산광역시 구별 스타벅스 매장수 구하기

In [14]:
sbuck_gu_counts= sbuck_df.pivot_table(index='구',
                                      values='매장명',
                                      aggfunc='count').rename(columns= {'매장명': '스타벅스_매장수'})
sbuck_gu_counts.head()

,스타벅스_매장수
구,
강서구,5
금정구,8
기장군,6
남구,6
동구,4


In [15]:
# busan_gu_df 수정
busan_gu_df = busan_gu_df[['do', 'city', 'latitude', 'longitude']]
columns = {'do':'도', 'city':'구', 'latitude':'위도', 'longitude':'경도'}
busan_gu_df.rename(columns=columns, inplace=True)
busan_gu_df.reset_index(drop=True, inplace=True)

In [16]:
busan_gu_df.head()

,도,구,위도,경도
0,부산시,강서구,35.209164,128.982908
1,부산시,금정구,35.240078,129.094319
2,부산시,남구,35.133408,129.086500
3,부산시,동구,35.135894,129.059175
4,부산시,동래구,35.201872,129.085856


In [17]:
busan_sgg = pd.merge(busan_gu_df, sbuck_gu_counts, how='left', on='구')
busan_sgg

,도,구,위도,경도,스타벅스_매장수
0,부산시,강서구,35.209164,128.982908,5
1,부산시,금정구,35.240078,129.094319,8
2,부산시,남구,35.133408,129.086500,6
3,부산시,동구,35.135894,129.059175,4
4,부산시,동래구,35.201872,129.085856,9
5,부산시,부산진구,35.159953,129.055319,19
6,부산시,북구,35.194181,128.992475,5
7,부산시,사상구,35.149467,128.993333,7
8,부산시,사하구,35.101428,128.977042,7
9,부산시,서구,35.094836,129.026378,4


In [27]:
busan_pop = pd.read_excel('./부산광역시_인구_2307.xlsx', usecols='A,S:T', header=2)
busan_pop.head()
columns = {'부산광역시':'구', 1561008: '세대수', 3364066: '인구수'}
busan_pop.rename(columns=columns, inplace=True)
busan_pop

,구,세대수,인구수
0,중구,24099,41862
1,서구,53670,107035
2,동구,46196,88013
3,영도구,54335,109774
4,부산진구,179744,360354
5,동래구,119963,274419
6,남구,118462,262385
7,북구,123573,279814
8,해운대구,170251,389843
9,사하구,141398,306547


In [28]:
busan_sgg = pd.merge(busan_sgg, busan_pop, how='left', on='구')
busan_sgg.head()

,도,구,위도,경도,스타벅스_매장수,세대수,인구수
0,부산시,강서구,35.209164,128.982908,5,62376,148176
1,부산시,금정구,35.240078,129.094319,8,106954,224488
2,부산시,남구,35.133408,129.086500,6,118462,262385
3,부산시,동구,35.135894,129.059175,4,46196,88013
4,부산시,동래구,35.201872,129.085856,9,119963,274419


In [34]:
# 사업체수
busan_biz = pd.read_excel('./부산광역시_사업체수_2307.xlsx', usecols='A:C', header=4)
columns = {'전체':'구', 401254:'사업체수', 1544504:'종사자수'}
busan_biz.rename(columns=columns, inplace=True)
busan_biz.head()

,구,사업체수,종사자수
0,중구,18306,69033
1,서구,11376,45728
2,동구,18208,76827
3,영도구,11616,46978
4,부산진구,44355,174317


In [35]:
busan_sgg_stat = pd.merge(busan_sgg, busan_biz, how='left', on='구')
busan_sgg_stat.tail()

,도,구,위도,경도,스타벅스_매장수,세대수,인구수,사업체수,종사자수
11,부산시,연제구,35.173186,129.082075,9,93650,204094,23946,96109
12,부산시,영도구,35.088117,129.070186,2,54335,109774,11616,46978
13,부산시,중구,35.103217,129.034508,6,24099,41862,18306,69033
14,부산시,해운대구,35.160019,129.165808,32,170251,389843,40946,154115
15,부산시,기장군,35.244775,129.222287,6,79179,181475,17930,77117


In [36]:
busan_sgg_stat.to_excel('./busan_sgg_stat.xlsx', index=False)

##### 데이터 시각화

In [37]:
sbuck_df.tail()

,매장명,위도,경도,매장타입,주소,구
132,해운대 엑스더스카이,35.159965,129.169756,general,부산광역시 해운대구 달맞이길30,해운대구
133,센텀신세계3F,35.169063,129.129066,general,부산광역시 해운대구 센텀남대로 35 (우동),해운대구
134,해운대중동역,35.167185,129.167886,general,부산광역시 해운대구 좌동순환로 6 (중동),해운대구
135,센텀몰1F,35.169810,129.127783,general,부산광역시 해운대구 센텀4로 15 (우동),해운대구
136,그랜드조선 부산,35.160034,129.163051,general,부산광역시 해운대구 해운대해변로 292 (중동),해운대구


In [39]:
sbuck_map = folium.Map(location=[35.189294, 129.061272], tiles='Stamen Terrain', zoom_start=11)

for idx in sbuck_df.index:
    lat = sbuck_df.loc[idx, '위도']
    lng = sbuck_df.loc[idx, '경도']
    name = sbuck_df.loc[idx, '매장명']

    folium.CircleMarker(location=[lat, lng], fill=True, fill_color='green', fill_opacity=1, popup=name,
                        color='yellow', weight=1, radius=3).add_to(sbuck_map)

sbuck_map

In [40]:
sbuck_map2 = folium.Map(location=[35.189294, 129.061272], tiles='Stamen Terrain', zoom_start=11)

for idx in sbuck_df.index:
    lat = sbuck_df.loc[idx, '위도']
    lng = sbuck_df.loc[idx, '경도']
    name = sbuck_df.loc[idx, '매장명']
    type = sbuck_df.loc[idx, '매장타입']

    # 매장타입별 색상
    fillcolor = ''
    size = 2
    if type == 'general':
        fillcolor = 'gray'
        size = 2
    elif type == 'reserve':
        fillcolor = 'black'
        size = 5
    elif type == 'generalDT':
        fillcolor = 'red'
        size = 3
    elif type == 'generalWT':
        fillcolor = 'blue'
        size = 3

    folium.CircleMarker(location=[lat, lng], fill=True, fill_color=fillcolor, fill_opacity=1, popup=name,
                        color='yellow', weight=1, radius=size).add_to(sbuck_map2)

sbuck_map2

In [50]:
# 시군구경계표시
sgg_geojson_path = './TL_SCCO_SIG.json'
busan_sgg_geo = json.load(open(sgg_geojson_path, encoding='utf-8'))

sbuck_map3 = folium.Map(location=[35.189294, 129.061272], tiles='CartoDB dark_matter', zoom_start=11)

folium.GeoJson(busan_sgg_geo, name='부산행정구역').add_to(sbuck_map3)

sbuck_map3

In [45]:
sbuck_mean = busan_sgg_stat['스타벅스_매장수'].mean()
sbuck_mean

8.5625

In [54]:
# 시군구경계표시
sgg_geojson_path = './TL_SCCO_SIG.json'
busan_sgg_geo = json.load(open(sgg_geojson_path, encoding='utf-8'))

sbuck_map3 = folium.Map(location=[35.189294, 129.061272], tiles='CartoDB dark_matter', zoom_start=11)

folium.GeoJson(busan_sgg_geo, name='부산행정구역').add_to(sbuck_map3)

for idx in busan_sgg_stat.index:
    lat = busan_sgg_stat.loc[idx, '위도']
    lng = busan_sgg_stat.loc[idx, '경도']
    count = busan_sgg_stat.loc[idx, '스타벅스_매장수']

    if count > sbuck_mean: # 구별 평균치보다 매장수가 많음
        fillcolor = 'red'
    else:
        fillcolor = 'blue'

    folium.CircleMarker(location=[lat, lng], color='#ffff00', fill_color=fillcolor, fill_opacity=0.7, weight=1.5, radius=count/2, popup=count).add_to(sbuck_map3)

sbuck_map3

In [55]:
# 시군구경계표시
sgg_geojson_path = './TL_SCCO_SIG.json'
busan_sgg_geo = json.load(open(sgg_geojson_path, encoding='utf-8'))

sbuck_map4 = folium.Map(location=[35.189294, 129.061272], tiles='CartoDB dark_matter', zoom_start=10)

folium.GeoJson(busan_sgg_geo, name='부산행정구역').add_to(sbuck_map4)

folium.Choropleth(
    geo_data=busan_sgg_geo,
    data=busan_sgg_stat,
    columns=['구', '스타벅스_매장수'],
    fill_color='Purples', # YlGn, RdYlGn, Spectral, Accent, Reds, Greens, Blues, Oranges, Purples, Greys, PuOr, OrRd
    fill_opacity=0.7, 
    line_opacity=0.5,
    key_on='properties.SIG_KOR_NM', # 이게 핵심 - key값이 없으면 지도에 색상표시 못함
).add_to(sbuck_map4)

sbuck_map4

In [57]:
# 부산시 인구수별 분포
busan_pop_choropleth = folium.Map(location=[35.189294, 129.061272], tiles='CartoDB dark_matter', zoom_start=10)

folium.Choropleth(geo_data=busan_sgg_geo, data=busan_sgg_stat,
                  columns=['구', '인구수'], fill_color='Purples', fill_opacity=0.7, line_opacity=0.5,
                  key_on='properties.SIG_KOR_NM').add_to(busan_pop_choropleth)

busan_pop_choropleth

- 결론 : 구에 살고 있는 인구수와 스타벅스 매장수는 관계 거의 없다

In [58]:
# 부산시 인구수별 분포
busan_biz_choropleth = folium.Map(location=[35.189294, 129.061272], tiles='CartoDB dark_matter', zoom_start=10)

folium.Choropleth(geo_data=busan_sgg_geo, data=busan_sgg_stat,
                  columns=['구', '사업체수'], fill_color='Purples', fill_opacity=0.7, line_opacity=0.5,
                  key_on='properties.SIG_KOR_NM').add_to(busan_biz_choropleth)

busan_biz_choropleth

- 결론 : 각 구별 사업체수와 스타벅스 매장수는 깊은 연관이 있다

         동작구는 사당역 근처 리저브 매장을 폐점했다(매출이 부진 예상)